# Hourly 유입량 Smoothing

## 1. 필요한 Python Library 읽기

In [ ]:
import pandas as pd
import datetime
from datetime import timedelta

## 2. Hourly 유입량 읽기

### 2.1 쿼리한 Hourly 유입량 읽어오기

In [ ]:
inflow_hourly_df_raw = pd.read_csv("sy_dam_hourly.csv")
inflow_hourly_df = inflow_hourly_df_raw.sort_values(by="OBSDH", ascending=True)
inflow_hourly_df.head()

### 2.2 쿼리의 DATE, TIME 포맷을 Pandas datetime 포맷으로 변환

In [ ]:
dt_series = []
for date in inflow_hourly_df["OBSDH"]:
    day = int(str(date)[6:8])
    time = int(str(date)[8:10])
    if int(str(date)[8:10]) == 24:
        time = 0
        datetimeobj = datetime.datetime(int(str(date)[0:4]), int(str(date)[4:6]), day, 0, 0)
        datetimeobj = datetimeobj + timedelta(days=1)
    else:
        datetimeobj = datetime.datetime(int(str(date)[0:4]), int(str(date)[4:6]), day, time, 0)
    dt_series.append(datetimeobj)
dt_series[0:4]

In [ ]:
# date의 날짜를 datetime 포맷으로 변경
inflow_hourly_df["date"] = dt_series
inflow_hourly_df.head()

In [ ]:
# Hourly 데이터의 시작 및 종료 날짜에 대해 정의하기
full_date = pd.DataFrame(
    {'date': pd.date_range(start='2000-01-01 01:00:00', end='2022-12-31 00:00:00', freq='1H'),
    }
)
full_date.head()

### 2.3 결측자료 확인

In [ ]:
# 데이터 Merge를 통해서 결측자료 확인
hourly_inflow_df = pd.merge(full_date, inflow_hourly_df, on="date", how="left")
hourly_inflow_df

In [ ]:
# 결측자료 갯수 확인
hourly_inflow_df.isna().sum().sum()

### 2.4 결측자료 Interpolation하기

In [ ]:
import numpy as np
hourly_inflow_df_1 = hourly_inflow_df.replace(0, np.NaN)
hourly_inflow_df_1

### 2.5 "0" 유입량 제거

* 연속적인 유입량 "0'값 이후 값을 가지게 되면 이전의 "0"값을 새로 계산된 값을 활용하여 Backfill로 평균값 적용

In [ ]:
inflow_values = hourly_inflow_df_1["IQTY"].bfill().div(
    hourly_inflow_df_1.groupby(hourly_inflow_df_1["IQTY"].notna()[::-1].cumsum())["IQTY"].transform('size')
)

In [ ]:
hourly_inflow_df_1["inflow(cms)"] = inflow_values.values
hourly_inflow_df_1

In [ ]:
hourly_inflow_df_1 = hourly_inflow_df_1.set_index("date")
hourly_inflow_df_1

### 2.6 Smoothing을 적용할 최종 유입량 값 취득

In [ ]:
hourly_inflow_filter = hourly_inflow_df_1[["inflow(cms)"]]
hourly_inflow_filter

## 3. Hourly 댐유입량에 Smoothing 적용 함수 정의

### 3.1 Smoothing 방법별 초기값 지정

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

### 3.2 Smoothing 분석 및 Plotting 함수 정의

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, RadioButtons

%matplotlib widget

def es_plot():
    
    #PARAMETERS
    alpha_h = 0.1
    beta_h = 0.1

    wint_season = 2
    wint_alpha = 0.2
    wint_beta = 0.2
    wint_zeta = 0.2

    def MovingAverage(og_fun, Window):

        if Window == 0:
            pass
            # mov_avg = np.zeros_like(og_fun)
        else:
            mov_avg = np.zeros_like(og_fun)
            for i in range(len(og_fun) - Window):
                mov_avg[i+Window] = og_fun[i:i+Window].sum()/Window

            return mov_avg

    def Centr_mov_avg(og_fun, Window):

        if Window == 1:
            pass
        else:
            HalfWindow = int((Window-1)/2)
            Smooth = np.zeros_like(og_fun)
            for i in range(HalfWindow, og_fun.shape[0]-HalfWindow):
                Smooth[i] = np.mean(og_fun[(i-HalfWindow):(i+HalfWindow)])

            return Smooth

    def ExpSmo_func(og_fun,alpha):

        if alpha<0:
            pass
        else:
            Smooth = np.zeros_like(og_fun)
            Smooth[0] = og_fun[0]

            for i in range(1,len(og_fun)):
                Smooth[i] = alpha * og_fun[i] + (1-alpha) * Smooth[i-1]

            return Smooth

    def Holt_exp_fun(og_fun,alpha,beta):

        if alpha<0 and beta<0:
            pass
        else:  
            #Initialization
            Smooth  = np.zeros_like(og_fun)
            Trend   = np.zeros_like(og_fun)
            Prediction = np.zeros_like(og_fun)

            Smooth[0] = og_fun[0]
            Smooth[1] = og_fun[1]
            Prediction[2] = Smooth[1] + (Smooth[1] - Smooth[0])

            for i in range(2,len(og_fun)-1):
                Smooth[i] = alpha*og_fun[i] + (1-alpha)*(Smooth[i-1]+Trend[i-1])
                Trend[i]  = beta*(Smooth[i]-Smooth[i-1]) + ((1-beta)*Trend[i-1])
                Prediction[i+1]  = Smooth[i]+Trend[i]


            return Prediction

    def Wint_exp_fun(og_fun, alpha, beta, zeta, S_len):

        if S_len == 0:
            pass
        else:

            Smooth     = np.zeros_like(og_fun)
            Trend      = np.zeros_like(og_fun)
            Season     = np.zeros_like(og_fun)
            Prediction = np.zeros_like(og_fun)

            #Initial values
            Season[:S_len] = og_fun[:S_len] / og_fun[:S_len].mean()
            Smooth[S_len]  = og_fun[S_len] / Season[0]
            Trend[S_len]   = Smooth[S_len] - (og_fun[S_len-1]/Season[S_len-1])

            for i in range(S_len, len(og_fun)-1):
                Season[i] = zeta * (og_fun[i]/Smooth[i]) + (1-zeta) * Season[i-S_len]
                Prediction[i+1] = (Smooth[i] + Trend[i]) * Season[i-S_len]
                Smooth[i+1] = alpha * (og_fun[i+1] / Season[i+1-S_len]) + (1 - alpha) * (Smooth[i] + Trend[i])
                Trend[i+1] = beta * (Smooth[i+1] - Smooth[i]) + (1-beta) * Trend[i]

            return Prediction

    def update_mov_avg(val):
        current_v = slider_mov_avg.val
        mov_avg = MovingAverage(y, Window=current_v)
        p1.set_ydata(mov_avg)
        fig.canvas.draw()

    def update_centr_mov_avg(val):
        current_v = slider_centr_mov_avg.val
        centrl_mov_avg = Centr_mov_avg(y, Window=current_v)
        p2.set_ydata(centrl_mov_avg)
        fig.canvas.draw()

    def update_exp_smoothing(val):
        current_v = slider_exp_smooth.val
        exp_smooth_value = ExpSmo_func(y, alpha=current_v)
        p3.set_ydata(exp_smooth_value)
        fig.canvas.draw()

    def update_holt_alpha(val):
        global alpha_h
        current_v = slider_holt_alpha.val
        alpha_h=current_v
        holt_value = Holt_exp_fun(y, alpha=current_v, beta=beta_h)
        p4.set_ydata(holt_value)
        fig.canvas.draw()

    def update_holt_beta(val):
        global beta_h
        current_v = slider_holt_beta.val
        beta_h = current_v
        holt_value = Holt_exp_fun(y, alpha=alpha_h, beta=current_v)
        p4.set_ydata(holt_value)
        fig.canvas.draw()

    def update_wint_season(val):
        global wint_season
        current_v = slider_wint_season.val
        wint_season = current_v
        wint_value = Wint_exp_fun(y, alpha=wint_alpha, beta=wint_beta, zeta=wint_zeta, S_len=current_v)
        p5.set_ydata(wint_value)
        fig.canvas.draw()

    def update_wint_alpha(val):
        global wint_alpha
        current_v = slider_wint_alpha.val
        wint_alpha = current_v
        wint_value = Wint_exp_fun(y, alpha=current_v, beta=wint_beta, zeta=wint_zeta, S_len=wint_season)
        p5.set_ydata(wint_value)
        fig.canvas.draw()

    def update_wint_beta(val):
        global wint_beta
        current_v = slider_wint_beta.val
        wint_beta = current_v
        wint_value = Wint_exp_fun(y, alpha=wint_alpha, beta=current_v, zeta=wint_zeta, S_len=wint_season)
        p5.set_ydata(wint_value)
        fig.canvas.draw()

    def update_wint_zeta(val):
        global wint_zeta
        current_v = slider_wint_zeta.val
        wint_zeta = current_v
        wint_value = Wint_exp_fun(y, alpha=wint_alpha, beta=wint_beta, zeta=current_v, S_len = wint_season)
        p5.set_ydata(wint_value)
        fig.canvas.draw()

       
    fig = plt.figure(figsize=[10, 9])
    ax  = fig.subplots()
    plt.title('Application of Smoothing methods to Remove Sawtooth Data from Hourly Inflow')

    mov_avg = MovingAverage(y, 3)
    centrl_mov_avg = Centr_mov_avg(y, 3)
    exp_smo = ExpSmo_func(y, 0.2)
    holt_exp = Holt_exp_fun(y,0.2,0.95)
    wint_exp = Wint_exp_fun(y, 0.2, 0.2, 0.2, 5)


    plt.subplots_adjust(bottom = 0.45)
    p     = ax.plot(x,y, label = 'Observational inflow', marker=".")
    p1,   = ax.plot(x,mov_avg,'r', label = 'Moving avg')
    p2,   = ax.plot(x,centrl_mov_avg,'g', label = 'Centralized moving avg')
    p3,   = ax.plot(x,exp_smo,'k', label = 'Exponential smoothing')
    p4,   = ax.plot(x,holt_exp,'y', label = "Holt's exponential smoothing")
    p5,   = ax.plot(x,wint_exp,'violet', label = "Winter's exponential smoothing")

    ax.legend()

    #Slider
    ax_slide  = plt.axes([0.2, 0.1, 0.65, 0.03])
    ax_slide2 = plt.axes([0.2, 0.14, 0.65, 0.03])
    ax_slide3 = plt.axes([0.2, 0.18, 0.65, 0.03])
    ax_slide4 = plt.axes([0.2, 0.22, 0.28, 0.03])
    ax_slide4_2 = plt.axes([0.57, 0.22, 0.28, 0.03])

    ax_slide5_1 = plt.axes([0.2, 0.26, 0.28, 0.03])
    ax_slide5_2 = plt.axes([0.57, 0.26, 0.28, 0.03])
    ax_slide5_3 = plt.axes([0.2, 0.30, 0.28, 0.03])
    ax_slide5_4 = plt.axes([0.57, 0.30, 0.28, 0.03])

   
    valinit = mov_avg_valinit
    slider_mov_avg = Slider(ax_slide, 'Window_size (Mov.avg)', valmin = 0, valmax=20, valinit=valinit, valstep=1, color='red', initcolor='none')
    valinit = centr_mov_avg_valinit    
    slider_centr_mov_avg = Slider(ax_slide2, 'Window size (C.Mov.avg)', valmin = 1, valmax=31, valinit=2, valstep=0.5, color='g',initcolor='none')
    valinit = exp_smooth_valinit
    slider_exp_smooth = Slider(ax_slide3, 'Exp_smoothing (α)', valmin = -0.03, valmax=1, valinit=0.2, valstep=0.03, color='k',initcolor='none')
    valinit = holt_alpha_valinit
    slider_holt_alpha = Slider(ax_slide4, "Holt's exp. α", valmin = -0.01, valmax=1, valinit=0.2, valstep=0.01, color='y',initcolor='none')
    valinit = holt_beta_valinit
    slider_holt_beta = Slider(ax_slide4_2, label='β', valmin = -0.01, valmax=1, valinit=0.95, valstep=0.01, color='y',initcolor='none')
    valinit = wint_season_valinit
    slider_wint_season = Slider(ax_slide5_1, label="Winter's season_length", valmin = 0, valmax=10, valinit=5, valstep=1, color='violet',initcolor='none')
    valinit = wint_alpha_valinit
    slider_wint_alpha  = Slider(ax_slide5_2, label='α', valmin = 0, valmax=1, valinit=0.2, valstep=0.01, color='violet',initcolor='none')
    valinit = wint_beta_valinit
    slider_wint_beta   = Slider(ax_slide5_3, label='β', valmin = 0, valmax=1, valinit=0.2, valstep=0.01, color='violet',initcolor='none')
    valinit = wint_zeta_valinit
    slider_wint_zeta   = Slider(ax_slide5_4, label='ζ', valmin = 0, valmax=1, valinit=0.2, valstep=0.01, color='violet',initcolor='none')

    slider_mov_avg.on_changed(update_mov_avg)
    slider_centr_mov_avg.on_changed(update_centr_mov_avg)
    slider_exp_smooth.on_changed(update_exp_smoothing)
    slider_holt_alpha.on_changed(update_holt_alpha)
    slider_holt_beta.on_changed(update_holt_beta)

    slider_wint_season.on_changed(update_wint_season)
    slider_wint_alpha.on_changed(update_wint_alpha)
    slider_wint_beta.on_changed(update_wint_beta)
    slider_wint_zeta.on_changed(update_wint_zeta)

    plt.show()
    
    return mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp

### 3.3 Performance 함수 정의

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

def performance_test(obs, sm_result):

    r2_result = r2_score(obs, sm_result)
    mse_result = mean_squared_error(obs, sm_result)
    mae_result = mean_absolute_error(obs, sm_result)
    rmse_result = mean_squared_error(obs, sm_result)**0.5

    return r2_result, mse_result, mae_result, rmse_result

## 4. Case별 Hourly 댐유입량에 Smoothing 방법 적용

 * Case는 매년 유입량이 가장 많았던 시기를 중심으로 일정기간의 데이터를 추출하여 Smoothing 적용

## Case-1

In [ ]:
plot1 = hourly_inflow_filter.loc["2001-07-13 0:00":"2001-07-17 0:00"]
x = plot1.index
y = plot1["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-2

In [ ]:
plot2 = hourly_inflow_filter.loc["2002-08-30 0:00":"2002-09-03 0:00"]
x = plot2.index
y = plot2["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-3

In [ ]:
plot3 = hourly_inflow_filter.loc["2003-09-11 0:00":"2003-09-15 0:00"]
x = plot3.index
y = plot3["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-4

In [ ]:
plot4 = hourly_inflow_filter.loc["2004-07-11 0:00":"2004-07-15 0:00"]
x = plot4.index
y = plot4["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-5

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

In [ ]:
plot5 = hourly_inflow_filter.loc["2005-06-25 0:00":"2005-06-29 0:00"]
x = plot5.index
y = plot5["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-6

In [ ]:
plot6 = hourly_inflow_filter.loc["2006-07-13 0:00":"2006-07-17 0:00"]
x = plot6.index
y = plot6["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-7

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

In [ ]:
plot7 = hourly_inflow_filter.loc["2007-08-08 0:00":"2007-08-12 0:00"]
x = plot7.index
y = plot7["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-8

In [ ]:
plot8 = hourly_inflow_filter.loc["2008-07-22 0:00":"2008-07-26 0:00"]
x = plot8.index
y = plot8["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-9

In [ ]:
plot9 = hourly_inflow_filter.loc["2009-08-10 0:00":"2009-08-14 0:00"]
x = plot9.index
y = plot9["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-10

In [ ]:
plot10 = hourly_inflow_filter.loc["2010-08-13 0:00":"2010-08-17 0:00"]
x = plot10.index
y = plot10["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-11

In [ ]:
plot11 = hourly_inflow_filter.loc["2011-07-25 0:00":"2011-07-29 0:00"]
x = plot11.index
y = plot11["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-12

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

In [ ]:
plot12 = hourly_inflow_filter.loc["2012-09-16 0:00":"2012-09-20 0:00"]
x = plot12.index
y = plot12["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-13

In [ ]:
plot13 = hourly_inflow_filter.loc["2013-07-12 0:00":"2013-07-16 0:00"]
x = plot13.index
y = plot13["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-14

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

In [ ]:
plot14 = hourly_inflow_filter.loc["2014-09-01 0:00":"2014-09-05 0:00"]
x = plot14.index
y = plot14["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-15

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

In [ ]:
plot15 = hourly_inflow_filter.loc["2015-08-23 0:00":"2015-08-27 0:00"]
x = plot15.index
y = plot15["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-16

In [ ]:
plot16 = hourly_inflow_filter.loc["2016-07-03 0:00":"2016-07-07 0:00"]
x = plot16.index
y = plot16["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-17

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

In [ ]:
plot17 = hourly_inflow_filter.loc["2017-07-01 0:00":"2017-07-05 0:00"]
x = plot17.index
y = plot17["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-18

In [ ]:
plot18 = hourly_inflow_filter.loc["2018-08-27 0:00":"2018-08-31 0:00"]
x = plot18.index
y = plot18["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-19

In [ ]:
mov_avg_valinit = 2
centr_mov_avg_valinit = 2
exp_smooth_valinit = 0.2
holt_alpha_valinit = 0.2
holt_beta_valinit = 0.95
wint_season_valinit = 5
wint_alpha_valinit = 0.2
wint_beta_valinit = 0.2
wint_zeta_valinit = 0.2

In [ ]:
plot19 = hourly_inflow_filter.loc["2019-07-24 0:00":"2019-07-28 0:00"]
x = plot19.index
y = plot19["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df

## Case-20

In [ ]:
plot20 = hourly_inflow_filter.loc["2020-09-01 0:00":"2020-09-05 0:00"]
x = plot20.index
y = plot20["inflow(cms)"]
mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp = es_plot()

In [ ]:
methods = [mov_avg, centrl_mov_avg, exp_smo, holt_exp, wint_exp]

r2_result_all = []
mse_result_all = []
mae_result_all = []
rmse_result_all = []

for i in methods:
    r2_result, mse_result, mae_result, rmse_result = performance_test(y.values, i)
    r2_result_all.append(r2_result)
    mse_result_all.append(mse_result)
    mae_result_all.append(mae_result)
    rmse_result_all.append(rmse_result)
    
evaluation = {"R2": r2_result_all,
              "MSE": mse_result_all,
              "MAE": mae_result_all,
              "RMSE": rmse_result_all,}
index = ["mov_avg", "centrl_mov_avg", "exp_smo", "holt_exp", "wint_exp"]
eval_df = pd.DataFrame(evaluation, index=index)
eval_df